# Define by run

In [1]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.3 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')

Training set shape: (537, 8)
Test set shape: (231, 8)


In [6]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [7]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [8]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-10-14 03:30:45,737] A new study created in memory with name: no-name-417e7ddf-94c6-44bc-a81d-ac518226fe68
[I 2024-10-14 03:30:49,739] Trial 0 finished with value: 0.7467411545623838 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 242, 'learning_rate': 0.021781018686482672, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.7467411545623838.
[I 2024-10-14 03:30:49,777] Trial 1 finished with value: 0.7579143389199255 and parameters: {'classifier': 'SVM', 'C': 0.3855946030604524, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 1 with value: 0.7579143389199255.
[I 2024-10-14 03:30:49,856] Trial 2 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 12.359830319703915, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 2 with value: 0.7858472998137801.
[I 2024-10-14 03:30:50,632] Trial 3 finished with value: 0.7672253258845437 and parameters: {'classifier': 'RandomForest', 'n_estimato

In [9]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 12.359830319703915, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7858472998137801


In [11]:
study.trials_dataframe().head()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.746741,2024-10-14 03:30:45.744811,2024-10-14 03:30:49.739285,0 days 00:00:03.994474,NaN,NaN,GradientBoosting,NaN,NaN,0.021781,15.0,6.0,6.0,242.0,COMPLETE
1,1,0.757914,2024-10-14 03:30:49.741092,2024-10-14 03:30:49.777468,0 days 00:00:00.036376,0.385595,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.785847,2024-10-14 03:30:49.779226,2024-10-14 03:30:49.856116,0 days 00:00:00.076890,12.359830,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.767225,2024-10-14 03:30:49.857876,2024-10-14 03:30:50.632085,0 days 00:00:00.774209,NaN,False,RandomForest,NaN,NaN,NaN,10.0,6.0,5.0,182.0,COMPLETE
4,4,0.757914,2024-10-14 03:30:50.633881,2024-10-14 03:30:50.954603,0 days 00:00:00.320722,NaN,True,RandomForest,NaN,NaN,NaN,6.0,7.0,5.0,62.0,COMPLETE


In [12]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,78
GradientBoosting,11
RandomForest,11


In [13]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.750973
RandomForest,0.766717
SVM,0.770878


In [14]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [15]:
# Optimization History
plot_optimization_history(study).show()

In [17]:
# Slice Plot
plot_slice(study).show()

Ref: https://optuna.org/ <br>
Thanks to : Campus X